# NN 
 
the idea is to initialize the network of DQNsb3_POMDP with the weights of the network of DQNsb3.

the only problem is that the network of DQNsb3_POMDP has addtional inputs

DQNMDP input = [pos,theta] \
DQNMDP input = [pos,belief]

In [1]:
import numpy as np

In [2]:
# maze size
N = 2

# thetas deformations (range(a,b),range(c,d))
l0 = 1
h0 = 10
l1 = 1
h1 = 10

maze = np.load(f"maze/maze_{N}.npy")

In [10]:
from stable_baselines3 import DQN
from environment.env import MDPGYMGridEnvDeform

MDP_env = MDPGYMGridEnvDeform(maze,l0,h0,l1,h1)
MDP_model = DQN.load("agents/pretrained/MDP/DQNsb3_z427ps8r.zip", env=MDP_env, device="cpu")

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/home/flaccagora/.miniconda3/envs/rob/lib/python3.9/site-packages/stable_baselines3/common/save_util.py:167: UserWarning: Could not deserialize object lr_schedule. Consider using `custom_objects` argument to replace this object.
Exception: code() takes at most 16 arguments (18 given)
  warnings.warn(
/home/flaccagora/.miniconda3/envs/rob/lib/python3.9/site-packages/stable_baselines3/common/save_util.py:167: UserWarning: Could not deserialize object exploration_schedule. Consider using `custom_objects` argument to replace this object.
Exception: code() takes at most 16 arguments (18 given)
  warnings.warn(


In [11]:
# model evaluation
from stable_baselines3.common.evaluation import evaluate_policy

mean_reward, std_reward = evaluate_policy(MDP_model, MDP_env, n_eval_episodes=10)
print(f"MDP model performance: {mean_reward} +/- {std_reward}")

MDP model performance: -9.4 +/- 5.85576638878294


credo di poter passare al modello la policy in forma di NN direttamnete anziché stringa "Multiinputpolicy"

In [12]:
from environment.env import POMDPGYMGridEnvDeform
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.monitor import Monitor

POMDP_env = POMDPGYMGridEnvDeform(maze,l0,h0,l1,h1, render_mode="rgb_array")

POMDP_model = DQN("MultiInputPolicy",POMDP_env,device="cpu")


In [13]:
# model evaluation
mean_reward, std_reward = evaluate_policy(POMDP_model, POMDP_env, n_eval_episodes=10)
print(f"POMDP model performance: {mean_reward} +/- {std_reward}")

POMDP model performance: -185.0 +/- 45.0


train the network to get $$POMDP\_model(pos,belief) = MDP\_model(pos,argmax(belief))$$

In [14]:

import copy

from typing import Dict, Tuple, Union

from stable_baselines3.common.preprocessing import is_image_space, maybe_transpose
from stable_baselines3.common.type_aliases import PyTorchObs
from stable_baselines3.common.utils import is_vectorized_observation, obs_as_tensor
from gymnasium import spaces

def obs_to_tensor(env, observation: Union[np.ndarray, Dict[str, np.ndarray]]) -> Tuple[PyTorchObs, bool]:
    """
    Convert an input observation to a PyTorch tensor that can be fed to a model.
    Includes sugar-coating to handle different observations (e.g. normalizing images).

    :param observation: the input observation
    :return: The observation as PyTorch tensor
        and whether the observation is vectorized or not
    """
    vectorized_env = False
    if isinstance(observation, dict):
        assert isinstance(
            env.observation_space, spaces.Dict
        ), f"The observation provided is a dict but the obs space is {env.observation_space}"
        # need to copy the dict as the dict in VecFrameStack will become a torch tensor
        observation = copy.deepcopy(observation)
        for key, obs in observation.items():
            obs_space = env.observation_space.spaces[key]
            if is_image_space(obs_space):
                obs_ = maybe_transpose(obs, obs_space)
            else:
                obs_ = np.array(obs)
            vectorized_env = vectorized_env or is_vectorized_observation(obs_, obs_space)
            # Add batch dimension if needed
            observation[key] = obs_.reshape((-1, *env.observation_space[key].shape))  # type: ignore[misc]

    elif is_image_space(env.observation_space):
        # Handle the different cases for images
        # as PyTorch use channel first format
        observation = maybe_transpose(observation, env.observation_space)

    else:
        observation = np.array(observation)

    if not isinstance(observation, dict):
        # Dict obs need to be handled separately
        vectorized_env = is_vectorized_observation(observation, env.observation_space)
        # Add batch dimension if needed
        observation = observation.reshape((-1, *env.observation_space.shape))  # type: ignore[misc]

    obs_tensor = obs_as_tensor(observation, 'cpu')
    return obs_tensor, vectorized_env


In [32]:
num_episodes = 10001

In [33]:
from collections import OrderedDict
import torch
from tqdm import trange

data = []

for episode in trange(num_episodes):
    s, _ = POMDP_env.reset()
   
    while True:
        mdp_s = OrderedDict([('x', s['x']),
                    ('y', s['y']),
                    ('phi', s['phi']),
                    ('theta', torch.tensor(MDP_env.deformations[torch.argmax(s['belief'])])),
                    ])

        a = MDP_model.policy.q_net(obs_to_tensor(MDP_env,mdp_s)[0])

        data.append((obs_to_tensor(POMDP_env,s)[0], a))

        s, _, terminated, truncated, info = POMDP_env.step(torch.argmax(a))

        if terminated or truncated:
            break
    # save data for training
    if  episode == 100 or episode == 1000 or episode == 10000:
        torch.save(data, f"POMDP_data_{num_episodes}.pt")


 49%|████▉     | 4922/10001 [11:15<11:37,  7.29it/s]  


KeyboardInterrupt: 

In [34]:
data = torch.load(f"POMDP_data_{10001}.pt", weights_only=True)

In [35]:
len(data)

24550

In [36]:
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, data):
        """
        Args:
            data (list of tuples): The dataset as a list of (x, y) tuples.
                                   x is a dictionary, and y is an array.
        """
        self.data = data

    def __len__(self):
        # Return the total number of samples
        return len(self.data)

    def __getitem__(self, idx):
        # Retrieve the sample at index `idx`
        x, y = self.data[idx]

        # Convert x (dictionary) to a tensor or process it as needed
        # Assuming all dictionary values are numerical and can be converted to tensors
        x_tensor = {key: value.clone().detach() for key, value in x.items()}

        # Convert y to a tensor
        y_tensor = y.clone().detach().squeeze()

        return x_tensor, y_tensor

def custom_collate_fn(batch):
    """
    Custom collate function to handle dictionaries in x.
    Args:
        batch: List of (x, y) tuples where x is a dictionary and y is a tensor.
    Returns:
        A dictionary of batched x values and a batched tensor for y.
    """
    # Separate the x and y values
    x_batch = [item[0] for item in batch]
    y_batch = [item[1] for item in batch]

    # Combine the dictionary values into batched tensors
    batched_x = {key: torch.stack([x[key] for x in x_batch]) for key in x_batch[0]}
    batched_y = torch.stack(y_batch)

    return batched_x, batched_y


In [37]:

# Create dataset and dataloader
dataset = CustomDataset(data)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True, collate_fn=custom_collate_fn)


In [38]:
import torch
from torch import nn
from torch.nn import functional as F

loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(POMDP_model.policy.q_net.parameters(), lr=1e-3)

for epoch in trange(10):
    for i, (batch_x, batch_y) in enumerate(dataloader):
        optimizer.zero_grad()
        output = POMDP_model.policy.q_net(batch_x)
        l = loss(output, batch_y)
        l.backward()
        optimizer.step()


100%|██████████| 10/10 [02:32<00:00, 15.22s/it]


In [42]:
mean_reward, std_reward = evaluate_policy(POMDP_model, POMDP_env, n_eval_episodes=10)
print(f"POMDP model performance: {mean_reward} +/- {std_reward}")

POMDP model performance: -170.0 +/- 60.0
